In [ ]:
import csv
from google.cloud import bigquery
from time import sleep

In [ ]:
def main():
    project_id = "driver-log-414519"
    client = bigquery.Client(project=project_id)
    csv_file_name = "matricula.csv"

    # Lista de anos que você quer processar
    anos = list(range(2007, 2024))  # Ajuste conforme a necessidade
    page_size = 50000
    print(f"Extraindo dados diretamente para: {csv_file_name}")
    with open(csv_file_name, mode="a", newline="", encoding="utf-8") as csv_file:
        writer = None
        for ano in anos:
            print(f"\nIniciando processamento do ano: {ano}")
            current_offset = 0
            block = 0
            while True:
                if (current_offset // 5300000) > block:
                    print(f"Esperando 60 segundos antes de continuar...")
                    sleep(60)
                    block += 1
                query = f"""
                    SELECT ano,
                        sigla_uf,
                        id_municipio,
                        rede,
                        id_aluno,
                        id_matricula,
                        idade,
                        sexo,
                        raca_cor,
                        necessidade_especial,
                        baixa_visao,
                        cegueira,
                        deficiencia_auditiva,
                        deficiencia_fisica,
                        deficiencia_intelectual,
                        surdez,
                        surdocegueira,
                        deficiencia_multipla,
                        superdotacao,
                        etapa_ensino,
                        especial_exclusiva,
                        regular,
                        eja,
                        profissionalizante
                    FROM `basedosdados.br_inep_censo_escolar.matricula`
                    WHERE ano = {ano}
                    LIMIT {page_size} OFFSET {current_offset}
                """
                print(f"Executando consulta para o ano {ano} com OFFSET {current_offset}")
                query_job = client.query(query)
                try:
                    rows = list(query_job.result())
                except:
                    print(f"Erro ao executar consulta para o ano {ano} com OFFSET {current_offset}")
                    sleep(600)
                    continue
                if not rows:
                    print(f"Concluído ano {ano}.")
                    break
                if writer is None:
                    # Escreve cabeçalho na primeira iteração
                    header = list(rows[0].keys())
                    writer = csv.writer(csv_file)
                    writer.writerow(header)
                for row in rows:
                    writer.writerow(row.values())
                current_offset += page_size
    print("\nExtração concluída com sucesso!")

In [ ]:
if __name__ == "__main__":
    main()